In [3]:
train_file = "/Users/test/Downloads/UPDATED_NLP_COURSE/06-Deep-Learning/train_qa.txt"

In [4]:
test_file = "/Users/test/Downloads/UPDATED_NLP_COURSE/06-Deep-Learning/test_qa.txt"

In [5]:
import pickle
import numpy as np

In [6]:
with open(train_file, 'rb') as f:
  train_data = pickle.load(f)

In [7]:
with open(test_file, 'rb') as f:
  test_data = pickle.load(f)

In [8]:
all_data = test_data + train_data

In [10]:
vocab = set()
for story, question, answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))

In [12]:
vocab.add("no")

In [13]:
vocab.add("yes")

In [14]:
vocab_len = len(vocab) +1

In [16]:
all_story_lens = [len(data[0]) for data in all_data]

In [17]:
max_story = max(all_story_lens)

In [18]:
max_question_len = max(len(data[1]) for data in all_data )

In [21]:
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [22]:
train_story_text = []
train_question_text = []
train_answers = []

In [24]:
for story, question, answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answers.append(answer)

In [25]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [27]:
def vectorize_stories(data, word_index= tokenizer.word_index, max_story_len=max_story, max_question_len=max_question_len ):
  
  X= []
  Xq = []
  Y = []
  
  for story, query, answer in data:
    x = [word_index[word.lower()] for word in story]
    xq = [word_index[word.lower()] for word in query]
    
    y = np.zeros(len(word_index) +1)
    
    X.append(x)
    Xq.append(xq)
    Y.append(y)
    

  return (pad_sequences(X, maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len),np.array(Y) )

In [28]:
inputs_train, queries_train, answer_train = vectorize_stories(train_data)

In [29]:
inputs_test, queries_test, answer_test = vectorize_stories(test_data)

In [30]:
inputs_test

array([[ 0,  0,  0, ..., 20,  9, 15],
       [ 0,  0,  0, ..., 20,  3, 15],
       [ 0,  0,  0, ..., 20,  3, 15],
       ...,
       [ 0,  0,  0, ..., 20, 26, 15],
       [ 0,  0,  0, ..., 20,  3, 15],
       [ 0,  0,  0, ..., 26, 25, 15]], dtype=int32)

In [32]:
answer_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
sum(answer_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [35]:
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate